<a href="https://colab.research.google.com/github/IsaacFigNewton/Taxonomic-Span-Categorization/blob/main/Test_Taxonomic_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [1]:
!pip install git+https://github.com/IsaacFigNewton/Taxonomic-Span-Categorization.git

  Cloning https://github.com/IsaacFigNewton/Taxonomic-Span-Categorization.git to /tmp/pip-req-build-qrohl0o6
  Running command git clone --filter=blob:none --quiet https://github.com/IsaacFigNewton/Taxonomic-Span-Categorization.git /tmp/pip-req-build-qrohl0o6
  Resolved https://github.com/IsaacFigNewton/Taxonomic-Span-Categorization.git to commit 05befb359c573b7098f8a7700431578b14e2566f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Config

In [2]:
import json
import spacy
from spacy import displacy
from tax_span_cat.SpanCategorizer import SpanCategorizer

In [3]:
nlp = spacy.load("en_core_web_sm")
ner = SpanCategorizer(taxonomic_features=["wordnet_synsets"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


# Test NER functionality

In [4]:
doc = nlp("In 1999, Tim Berners-Lee was one of the first to introduce the idea of the Semantic Web.")
ner_doc = ner(doc)
ner_doc.ents

(1999, Tim Berners-Lee, one, first, the idea, the Semantic Web)

In [5]:
# TODO: find out why this breaks
json.dumps(ner_doc.spans, indent=4, default=str)

'"{\'ENTITY\': [Tim Berners-Lee, the idea, the Semantic Web]}"'

In [7]:
displacy.render(ner_doc, style="ent", jupyter=True)